In [2]:
import pandas as pd

In [ ]:
matches = pd.read_csv("matches.csv", index_col=0)
matches.head()
matches.shape

In [ ]:
matches["team"].value_counts()

In [ ]:
matches["round"].value_counts()

In [ ]:
matches.dtypes

In [ ]:
matches["date"] = pd.to_datetime(matches["date"])
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["result"] == "W").astype("int")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
train = matches[matches["date"] < '2024-01-01']
test = matches[matches["date"] > '2024-01-01']

In [ ]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]
rf.fit(train[predictors], train["target"])
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score # look at precentage of the time your prediction was correct (Method 1)
acc = accuracy_score(test["target"], preds)
acc

In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

In [ ]:
from sklearn.metrics import precision_score # when predict win, what percentage of the time did team actually win? (Method 2)
precision_score(test["target"], preds) 

In [ ]:
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City")
group

In [ ]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
rolling_averages(group, cols, new_cols)

In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

In [ ]:
def make_predictions(data, predictors):
    train = matches[matches["date"] < '2024-01-01']
    test = matches[matches["date"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index =test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)
precision
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)